# Hate speech detection with CLL

#### Inspired in codes provided by Chris McCormick and Nick Ryan

In [ ]:
import tensorflow as tf
import torch

device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
!pip install transformers

# LOAD DATA

In [ ]:
import pandas as pd
df_full = #pd.read_csv()
pt_df_train = #pd.read_csv()

In [ ]:
#When using JL, CL/JL and CL/JL+ strategies, we add a part of the data in Target Language on initial training
percentage = 0.3

In [ ]:
amostra = pt_df_train.sample(frac=percentage)

In [ ]:
amostraValid = amostra.sample(frac=0.2)

In [ ]:
amostra = amostra.drop(amostraValid.index)

In [ ]:
amostraValidIt = df_full.sample(frac=0.1)

In [ ]:
df_full = df_full.drop(amostraValidIt.index)

In [ ]:
df_full = df_full.append(amostra)

In [ ]:
from transformers import BertTokenizer

# Loading BERTimbau tokenizer
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=True)

In [ ]:
# Tokenizing the sentences 
max_len = 128
input_ids = []
attention_masks = []
labels = []

for index, row in df_full.iterrows():  
    text = row['translation']
    encoded_dict = tokenizer.encode_plus(text, 
                                              max_length=max_len, 
                                              padding='max_length',
                                              truncation=True, 
                                              return_tensors='pt')
    
  
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    labels.append(row['target'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [ ]:
from torch.utils.data import TensorDataset, random_split

train_dataset = TensorDataset(input_ids, attention_masks, labels)
df_valid = amostraValid.append(amostraValidIt)

In [ ]:
import torch
torch.cuda.empty_cache()

max_len = 128

labels_val = []
input_ids_val = []
attn_masks_val = []

for index, row in df_valid.iterrows():


    text = row['translation']
    encoded_dict = tokenizer.encode_plus(text, 
                                              max_length=max_len, 
                                              padding='max_length',
                                              truncation=True, 
                                              return_tensors='pt')


    input_ids_val.append(encoded_dict['input_ids'])
    attn_masks_val.append(encoded_dict['attention_mask'])
    labels_val.append(row['target'])

    
input_ids_val = torch.cat(input_ids_val, dim=0)
attn_masks_val = torch.cat(attn_masks_val, dim=0)
labels_val = torch.tensor(labels_val)

In [ ]:
val_dataset = TensorDataset(input_ids_val, attn_masks_val, labels_val)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "neuralmind/bert-large-portuguese-cased", 
    num_labels = 2, output_attentions = False, output_hidden_states = False,)

model.cuda()

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-6, 
                  eps = 1e-8 
                )


In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


## Training Loop

In [ ]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            

            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits


        total_train_loss += loss.item()

        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


        optimizer.step()

        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================


    print("")
    print("Running Validation...")

    t0 = time.time()


    model.eval()

 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0


    for batch in validation_dataloader:
        

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():        

            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)


        loss = result.loss
        logits = result.logits
            

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

import pandas as pd

pd.set_option('precision', 2)

df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
df_stats

sns.set(style='darkgrid')

sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

# Saving Model (executed when using CL, CL/JL or CL/JL+ strategies)

In [ ]:
import os 
import time
from datetime import datetime

timestamp = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H%M')

local_dir = './Model Save ' + timestamp + '/'

if not os.path.exists(local_dir):
    os.makedirs(local_dir)

model.save_pretrained(local_dir)

In [ ]:
import shutil
#copy to gdrive
gdrive_dir = #'/location'

gdrive_dir = gdrive_dir + local_dir[2:]

shutil.copytree(local_dir, gdrive_dir)

Copying model save to ../content/drive/My Drive/Colab Notebooks/multi/Model Save 2022-02-03 2021/
     config.json                      0.00 MB
     pytorch_model.bin             1275.78 MB


# **Model Loading**

### After we save the model, we then load the weights onto a new model, and then we start new fine tunings (executed when using CL, CL/JL and CL/JL+ strategies)

In [ ]:
from transformers import XLMRobertaForSequenceClassification
import torch

device = torch.device('cuda')
desc = model.to(device)
torch.cuda.empty_cache()
gdrive_dir = #'/location'
model = BertForSequenceClassification.from_pretrained(gdrive_dir, num_labels=2)

model.cuda()

Loading model to GPU...
  GPU: Tesla T4
    DONE.


# **ADDITIONAL FINE TUNINGS**

### Executed when using CL/JL+ strategy

In [ ]:
pt_df_train = pt_df_train.drop(amostra.index)

In [ ]:
#determine the number of additional fine tunings
NUM_FINE = 5

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=NUM_FINE)

In [ ]:
samples = skf.split(pt_df_train['text'], pt_df_train['target'])

# Additional fine-tunings beginning

In [ ]:
ITERATION_NUMBER = 1
generator =next(samples)
kfold = list(generator)

amostraTrainPtX = pt_df_train.iloc[kfold[0],:]
amostraTestPtx = pt_df_train.iloc[kfold[1],:]

amostraTreinoPt = amostraTrainPtX
amostraTestePt =  amostraTestPtx

In [ ]:
import torch

#tokenizing the training dataset
max_len = 128

labels = []
input_ids = []
attention_masks = []


for index, row in amostraTreinoPt.iterrows():

    text = row['text']

    encoded_dict = tokenizer.encode_plus(text, 
                                              max_length=max_len, 
                                              padding='max_length',
                                              truncation=True, 
                                              return_tensors='pt')

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    labels.append(row['target'])


input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)


In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
torch.cuda.empty_cache()

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


batch_size = 16

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size 
        )


validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size 
        )

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),
                  lr = 5e-6, 
                  eps = 1e-8
                )

In [ ]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
from transformers import get_linear_schedule_with_warmup
epochs = 3

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
import random
import numpy as np
torch.cuda.empty_cache()



seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


training_stats = []


total_t0 = time.time()

for epoch_i in range(0, epochs):
    

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()


    total_train_loss = 0


    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

     
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

      
        model.zero_grad()        

       
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

       
        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            

    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        
      
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
      
        with torch.no_grad():        


            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        loss = result.loss
        logits = result.logits
            
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        


    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

import pandas as pd

pd.set_option('precision', 2)

df_stats = pd.DataFrame(data=training_stats)

df_stats = df_stats.set_index('epoch')

df_stats

sns.set(style='darkgrid')

sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

In [ ]:
import torch

labels_ar = []
input_ids_ar = []
attn_masks_ar = []

print('Encoding test examples...')

for index, ex in amostraTestePt.iterrows():

    text = ex['text']
    encoded_dict = tokenizer.encode_plus(text, 
                                              max_length=max_len, 
                                              padding='max_length',
                                              truncation=True, 
                                              return_tensors='pt')

    input_ids_ar.append(encoded_dict['input_ids'])
    attn_masks_ar.append(encoded_dict['attention_mask'])
    labels_ar.append(ex['target'])
    
input_ids_ar = torch.cat(input_ids_ar, dim=0)
attn_masks_ar = torch.cat(attn_masks_ar, dim=0)

labels_ar = torch.tensor(labels_ar)

print('   DONE. {:,} examples.'.format(len(labels_ar)))

## Evaluating the model with data from target language


In [ ]:
prediction_dataset = TensorDataset(input_ids_ar, attn_masks_ar, labels_ar)

prediction_dataloader = DataLoader(prediction_dataset, batch_size=batch_size)


In [ ]:
#evaluating model
model.eval()

predictions , true_labels = [], []

count = 0

for batch in prediction_dataloader:
    
    batch = tuple(t.to(device) for t in batch)
    
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        outputs = model(b_input_ids, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    predictions.append(logits)
    true_labels.append(label_ids)



In [ ]:
flat_predictions = np.concatenate(predictions, axis=0)
flat_true_labels = np.concatenate(true_labels, axis=0)

predicted_labels = np.argmax(flat_predictions, axis=1).flatten()

In [ ]:
#printing f1 weighted
from sklearn.metrics import f1_score
f1_score(flat_true_labels, predicted_labels, average='weighted')

In [ ]:
 #printing classification report
 from sklearn.metrics import classification_report
 print(classification_report(flat_true_labels, predicted_labels))

# Additional fine-tunings ending